# Introduction

## Objectif du Notebook

L'objectif de ce notebook est de développer et d'évaluer des modèles d'analyse de sentiments en utilisant **BERT** (Bidirectional Encoder Representations from Transformers), tout en comparant différentes versions des données textuelles : avec et sans pré-traitements (lemmatisation et stemming). BERT, étant un modèle pré-entraîné puissant, nous permet de capturer les relations contextuelles entre les mots dans un texte. En parallèle, nous allons comparer les performances des modèles CNN et LSTM entraînés sur des textes bruts et pré-traités.

## Présentation de la Méthode BERT

BERT est un modèle de traitement du langage naturel basé sur une architecture de **Transformers**, permettant de capturer de manière bidirectionnelle les relations entre les mots. BERT est particulièrement efficace pour des tâches telles que l'analyse de sentiments car il tient compte du contexte des mots dans les phrases, contrairement aux modèles classiques comme Word2Vec qui sont unidirectionnels.

### BERT (Bidirectional Encoder Representations from Transformers)

BERT prend en compte le contexte avant et après un mot pour créer des représentations contextuelles riches. C'est pourquoi il est très performant pour des tâches de compréhension sémantique, comme la classification de sentiments, où l'interprétation d'un mot dépend de son contexte global.

### Comparaison avec et sans Pré-traitements (Lemmatisation et Stemming)

Dans ce notebook, nous utiliserons à la fois :
- **Les textes pré-traités** (lemmatisation et stemming) : Ces techniques consistent à ramener les mots à leur forme de base ou à leur racine pour réduire la variabilité linguistique. Cela permet de simplifier les phrases avant de les soumettre à BERT.
- **Les textes bruts** : Sans aucun pré-traitement linguistique, laissant à BERT la capacité d'analyser les mots dans leur forme originale et de capturer les nuances du langage.

L'objectif est de comparer l'impact du pré-traitement sur les performances du modèle BERT.

## Plan du Notebook

1. **Chargement et Préparation des Données** : Nous allons charger les données, les nettoyer, les tokeniser, et les préparer sous trois formats : brut, lemmatisé et stemmé.
2. **Tokenisation avec BERT** : Nous utiliserons le tokenizer BERT pour convertir les trois versions de textes en tokens adaptés au modèle BERT.
3. **Utilisation de BERT** : Nous extrairons des embeddings contextuels à partir du modèle BERT pour c
4. **Définition du modèle CNN** : Ici nous définissons le modèle CNN, avec une gridSearch et une méthode Mlflow. chaque version de du modèle CNN. **EICI, nous allons entrainer le modeles de CNN .ersi**Définition du modèle LSTM** : Ici nous définissons le modèle LSTM, avec une gridSearch et une méthode Mlflow.
7. **Entraînement du modèle LSTM** : ICI, nous allons entrainer le modeles de LSTM .re dans une API de prédiction.


# 1. Importation des Bibliothèques

In [9]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, TFBertModel


In [11]:
# Charger les données (en supposant que vous avez déjà les colonnes nettoyées)
data = pd.read_csv('../data/database_p7_rework.csv')

# Transformation des labels : 0 reste 0 et 4 devient 1
data['target_binary'] = data['target'].apply(lambda x: 0 if x == 0 else 1)

# Vérification des transformations
print(data['target_binary'].value_counts())

# Ensuite, définissez y comme suit :
y = data['target_binary']

0    800000
1    798315
Name: target_binary, dtype: int64


In [25]:
# Sélection d'un échantillon équilibré de 16 000 données (8 000 par classe)
sample_data = data.groupby('target_binary', group_keys=False).apply(lambda x: x.sample(8000, random_state=42))

# 2. Importation & Tokénization avec BERT

In [54]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import time

# Initialiser le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Utiliser le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fonction pour encoder des textes en utilisant BERT avec batching et un timer
def encode_with_bert_batch(texts, batch_size=32):
    all_embeddings = []
    total_batches = len(texts) // batch_size + int(len(texts) % batch_size > 0)
    
    start_time = time.time()
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Chronométrage pour le lot actuel
        batch_start_time = time.time()
        
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=25).to(device)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.cpu().detach().numpy()
        all_embeddings.extend(embeddings)
        
        # Calcul du temps écoulé pour ce lot
        batch_time = time.time() - batch_start_time
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / (i + batch_size)) * len(texts)
        remaining_time = estimated_total_time - elapsed_time
        
        print(f"Lot {i//batch_size + 1}/{total_batches} traité en {batch_time:.2f} secondes. Temps restant estimé : {remaining_time:.2f} secondes.")
    
    return np.array(all_embeddings)



A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [56]:
# Appliquer BERT sur les descriptions lemmatisées avec batching et timer
bert_features = encode_with_bert_batch(sample_data['text_cleaned'].tolist())

Lot 1/500 traité en 0.54 secondes. Temps restant estimé : 270.63 secondes.
Lot 2/500 traité en 0.44 secondes. Temps restant estimé : 243.38 secondes.
Lot 3/500 traité en 0.39 secondes. Temps restant estimé : 226.32 secondes.
Lot 4/500 traité en 0.37 secondes. Temps restant estimé : 215.53 secondes.
Lot 5/500 traité en 0.35 secondes. Temps restant estimé : 206.99 secondes.
Lot 6/500 traité en 0.39 secondes. Temps restant estimé : 204.39 secondes.
Lot 7/500 traité en 0.38 secondes. Temps restant estimé : 201.36 secondes.
Lot 8/500 traité en 0.38 secondes. Temps restant estimé : 199.23 secondes.
Lot 9/500 traité en 0.41 secondes. Temps restant estimé : 199.31 secondes.
Lot 10/500 traité en 0.40 secondes. Temps restant estimé : 198.56 secondes.
Lot 11/500 traité en 0.46 secondes. Temps restant estimé : 200.62 secondes.
Lot 12/500 traité en 0.42 secondes. Temps restant estimé : 200.46 secondes.
Lot 13/500 traité en 0.46 secondes. Temps restant estimé : 201.97 secondes.
Lot 14/500 traité en 

In [57]:
# Séparation des données en ensembles d'entraînement et de test pour les données lemmatisées
X_train, X_test, y_train, y_test = train_test_split(
    bert_features, sample_data['target_binary'], test_size=0.2, random_state=42)

# 3. Construction du modèle CNN 

In [59]:
import mlflow
import mlflow.keras
import time
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid

# Fonction pour créer le modèle CNN
def create_cnn_model(max_length=100, num_filters=128, kernel_size=5, dropout_rate=0.2, activation_type='relu', embedding_dim=768):
    model = Sequential()

    # Couche de convolution et normalisation
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(25, embedding_dim)))

    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    
    # Ajout de l'activation dynamique dans les couches denses
    if activation_type == 'relu':
        model.add(Dense(128, activation='relu'))
    elif activation_type == 'leaky_relu':
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.1))
    elif activation_type == 'prelu':
        model.add(Dense(128))
        model.add(PReLU())

    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Fonction pour entraîner et loguer un modèle CNN avec GridSearch et les activations avancées
def train_and_log_cnn(X_train, y_train, X_test, y_test, experiment_name, param_grid, max_length=100, embedding_dim=768):
    mlflow.set_experiment(experiment_name)
    
    best_model = None
    best_accuracy = 0
    best_params = None

    # Parcourir chaque combinaison d'hyperparamètres
    for params in ParameterGrid(param_grid):
        num_filters = params['num_filters']
        kernel_size = params['kernel_size']
        dropout_rate = params['dropout_rate']
        activation_type = params['activation_type']

        with mlflow.start_run(run_name=f"CNN_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}_activation={activation_type}"):

            # Créer le modèle CNN avec les hyperparamètres courants
            model = create_cnn_model(max_length=max_length, num_filters=num_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, activation_type=activation_type, embedding_dim=embedding_dim)

            # Early stopping pour éviter l'overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=4)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

            # Entraîner le modèle
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=1)
            training_time = time.time() - start_time

            # Prédictions et évaluation
            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            y_pred_proba = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc_score = roc_auc_score(y_test, y_pred_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            # Matrice de confusion
            conf_matrix = confusion_matrix(y_test, y_pred)

            # Loguer les hyperparamètres dans MLFlow
            mlflow.log_param("num_filters", num_filters)
            mlflow.log_param("kernel_size", kernel_size)
            mlflow.log_param("dropout_rate", dropout_rate)
            mlflow.log_param("activation_type", activation_type)

            # Loguer les métriques dans MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("auc", auc_score)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("training_time", training_time)

            # Sauvegarder le modèle avec MLFlow
            mlflow.keras.log_model(model, f"cnn_model_{num_filters}_{kernel_size}_{dropout_rate}")

            # Sauvegarder et loguer la matrice de confusion
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.xlabel('Prédictions')
            plt.ylabel('Vérités')
            plt.title(f"Matrice de Confusion - CNN")
            conf_matrix_path = f"./matrice/confusion_matrix_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(conf_matrix_path)
            mlflow.log_artifact(conf_matrix_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Sauvegarder et loguer la courbe ROC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            plt.figure()
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_score:.2f})")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend(loc="best")
            roc_curve_path = f"./matrice/roc_curve_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(roc_curve_path)
            mlflow.log_artifact(roc_curve_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Comparer pour garder le meilleur modèle
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
                best_model = model

    print(f"Meilleurs paramètres : {best_params} avec une accuracy de {best_accuracy:.4f}")

    # Retourner le meilleur modèle
    return best_model, best_params

# GridSearch pour le modèle CNN avec les hyperparamètres supplémentaires
param_grid_cnn = {
    'num_filters': [64, 128, 256],   # Filtres à tester
    'kernel_size': [3, 5],        # Tailles de kernel à tester
    'dropout_rate': [0.2, 0.5],      # Dropout à tester
    'activation_type': ['relu', 'leaky_relu', 'prelu']  # Activations à tester
}

# 4. Entrainement du modèle CNN

In [62]:
# Entraînement du modèle CNN avec GridSearch sur les embeddings BERT
best_model, best_params = train_and_log_cnn(
    X_train, y_train, X_test, y_test, 
    experiment_name="BERT_CNN_Experiment", 
    param_grid=param_grid_cnn, 
    max_length=100, 
    embedding_dim=768  # Embedding dim from BERT
)

print(f"Meilleur modèle entraîné avec les paramètres : {best_params}")

Epoch 1/20
200/200 [==============================] - 3s 10ms/step - loss: 0.5537 - accuracy: 0.7205 - val_loss: 0.4906 - val_accuracy: 0.7678 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 2s 9ms/step - loss: 0.4540 - accuracy: 0.7911 - val_loss: 0.4656 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 2s 9ms/step - loss: 0.3995 - accuracy: 0.8207 - val_loss: 0.4698 - val_accuracy: 0.7769 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 2s 10ms/step - loss: 0.3481 - accuracy: 0.8484 - val_loss: 0.5097 - val_accuracy: 0.7812 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 2s 9ms/step - loss: 0.2240 - accuracy: 0.9104 - val_loss: 0.6054 - val_accuracy: 0.7688 - lr: 2.0000e-04
Epoch 6/20
100/100 [==============================] - 0s 2ms/step


2024/10/13 07:56:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmprgh8j560\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmprgh8j560\model\data\model\assets
2024/10/13 07:56:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 1/20
200/200 [==============================] - 3s 13ms/step - loss: 0.5708 - accuracy: 0.7120 - val_loss: 0.4804 - val_accuracy: 0.7700 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 3s 13ms/step - loss: 0.4611 - accuracy: 0.7873 - val_loss: 0.4731 - val_accuracy: 0.7812 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 3s 13ms/step - loss: 0.4055 - accuracy: 0.8198 - val_loss: 0.4709 - val_accuracy: 0.7763 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 3s 13ms/step - loss: 0.3592 - accuracy: 0.8397 - val_loss: 0.5193 - val_accuracy: 0.7638 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 2s 12ms/step - loss: 0.2966 - accuracy: 0.8723 - val_loss: 0.5579 - val_accuracy: 0.7641 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 3s 13ms/step - loss: 0.1637 - accuracy: 0.9373 - val_loss: 0.6753 - val_accuracy: 0.7734 - lr: 2.0000e-04
Epoch 7/20
100/100 [==============================] - 0s 2ms

...

Epoch 1/20
200/200 [==============================] - 7s 29ms/step - loss: 0.6405 - accuracy: 0.6702 - val_loss: 0.5040 - val_accuracy: 0.7638 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 6s 30ms/step - loss: 0.4999 - accuracy: 0.7607 - val_loss: 0.5089 - val_accuracy: 0.7472 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 6s 30ms/step - loss: 0.4538 - accuracy: 0.7937 - val_loss: 0.4760 - val_accuracy: 0.7719 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 6s 31ms/step - loss: 0.4165 - accuracy: 0.8127 - val_loss: 0.4969 - val_accuracy: 0.7603 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 6s 30ms/step - loss: 0.3675 - accuracy: 0.8396 - val_loss: 0.5273 - val_accuracy: 0.7681 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 6s 30ms/step - loss: 0.2511 - accuracy: 0.8980 - val_loss: 0.5896 - val_accuracy: 0.7734 - lr: 2.0000e-04
Epoch 7/20
100/100 [==============================] - 0s 5ms

2024/10/13 08:16:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp2li4g0te\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp2li4g0te\model\data\model\assets
2024/10/13 08:17:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'prelu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 128} avec une accuracy de 0.7791
Meilleur modèle entraîné avec les paramètres : {'activation_type': 'prelu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 128}


# 5. Construction du modèle LSTM

In [84]:
import mlflow
import mlflow.keras
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, LeakyReLU, PReLU,  Bidirectional
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid



# Fonction pour créer un modèle LSTM
def create_lstm_model(input_shape, lstm_units=128, dropout_rate=0.2, activation_type='relu'):
    model = Sequential()

    # Couche LSTM simple
    model.add((Bidirectional(LSTM(lstm_units, return_sequences=False, input_shape=input_shape))))
    
    # Batch Normalization après la couche LSTM
    model.add(BatchNormalization())

    # Activation dynamique ou fixe
    if activation_type == 'leakyrelu':
        model.add(LeakyReLU())
    elif activation_type == 'prelu':
        model.add(PReLU())
    else:
        model.add(Dense(128, activation=activation_type))

    # Dropout pour éviter le surapprentissage
    model.add(Dropout(dropout_rate))
    
    # Couche Dense
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    
    # Couche de sortie
    model.add(Dense(1, activation='sigmoid'))

    # Compilation du modèle
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Fonction pour entraîner et loguer un modèle LSTM avec GridSearch et les métriques avancées
def train_and_log_lstm(X_train, y_train, X_test, y_test, experiment_name, param_grid, input_shape):
    mlflow.set_experiment(experiment_name)
    
    best_model = None
    best_accuracy = 0
    best_params = None

    # Parcourir chaque combinaison d'hyperparamètres
    for params in ParameterGrid(param_grid):
        lstm_units = params['lstm_units']
        dropout_rate = params['dropout_rate']
        activation_type = params['activation_type']

        with mlflow.start_run(run_name=f"LSTM_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}"):

            # Créer le modèle LSTM avec les hyperparamètres courants
            model = create_lstm_model(input_shape=input_shape, lstm_units=lstm_units, dropout_rate=dropout_rate, activation_type=activation_type)

            # Early stopping pour éviter l'overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=4)

            # Réduction du taux d'apprentissage lorsque la validation stagne
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

            # Entraîner le modèle
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=1)
            training_time = time.time() - start_time

            # Prédictions et évaluation
            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            y_pred_proba = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc_score = roc_auc_score(y_test, y_pred_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            # Matrice de confusion
            conf_matrix = confusion_matrix(y_test, y_pred)

            # Loguer les hyperparamètres dans MLFlow
            mlflow.log_param("lstm_units", lstm_units)
            mlflow.log_param("dropout_rate", dropout_rate)
            mlflow.log_param("activation_type", activation_type)

            # Loguer les métriques dans MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("auc", auc_score)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("training_time", training_time)

            # Sauvegarder le modèle avec MLFlow
            mlflow.keras.log_model(model, f"lstm_model_{lstm_units}_{dropout_rate}_{activation_type}")

            # Sauvegarder et loguer la matrice de confusion
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.xlabel('Prédictions')
            plt.ylabel('Vérités')
            plt.title(f"Matrice de Confusion - LSTM")
            conf_matrix_path = f"./matrice/confusion_matrix_lstm_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}.png"
            plt.savefig(conf_matrix_path)
            mlflow.log_artifact(conf_matrix_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook
            
            # Sauvegarder et loguer la courbe ROC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            plt.figure()
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_score:.2f})")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend(loc="best")
            roc_curve_path = f"./matrice/roc_curve_lstm_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}.png"
            plt.savefig(roc_curve_path)
            mlflow.log_artifact(roc_curve_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Comparer pour garder le meilleur modèle
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
                best_model = model

    print(f"Meilleurs paramètres : {best_params} avec une accuracy de {best_accuracy:.4f}")

    # Retourner le meilleur modèle
    return best_model, best_params

# GridSearch pour le modèle LSTM avec les hyperparamètres
param_grid_lstm = {
    'lstm_units': [64, 128, 256],      # Nombre de cellules LSTM
    'dropout_rate': [0.2, 0.5],        # Taux de Dropout
    'activation_type': ['relu', 'leakyrelu', 'prelu']  # Type d'activation
}



# 6. entrainement du modèle LSTM

In [86]:
# Entraînement du modèle LSTM avec GridSearch sur les embeddings BERT
best_model, best_params = train_and_log_lstm(
    X_train, y_train, X_test, y_test, 
    experiment_name="BERT_LSTM_Experiment", 
    param_grid=param_grid_lstm, 
    input_shape=(25, 768)  # Correspond à la longueur des séquences et à la dimension des embeddings BERT
)

print(f"Meilleur modèle entraîné avec les paramètres : {best_params}")


Epoch 1/20
200/200 [==============================] - 44s 203ms/step - loss: 0.5486 - accuracy: 0.7193 - val_loss: 0.5547 - val_accuracy: 0.7231 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 56s 280ms/step - loss: 0.4770 - accuracy: 0.7718 - val_loss: 0.5347 - val_accuracy: 0.7203 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 55s 275ms/step - loss: 0.4412 - accuracy: 0.7948 - val_loss: 0.4901 - val_accuracy: 0.7700 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 54s 272ms/step - loss: 0.3950 - accuracy: 0.8157 - val_loss: 0.4871 - val_accuracy: 0.7616 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 55s 274ms/step - loss: 0.3427 - accuracy: 0.8495 - val_loss: 0.5696 - val_accuracy: 0.7609 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 55s 274ms/step - loss: 0.2756 - accuracy: 0.8805 - val_loss: 0.6291 - val_accuracy: 0.7462 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/13 10:21:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp_lmbomt8\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp_lmbomt8\model\data\model\assets
2024/10/13 10:22:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 1/20
200/200 [==============================] - 72s 342ms/step - loss: 0.5564 - accuracy: 0.7197 - val_loss: 0.5349 - val_accuracy: 0.7459 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 67s 337ms/step - loss: 0.4804 - accuracy: 0.7741 - val_loss: 0.5488 - val_accuracy: 0.6953 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 68s 338ms/step - loss: 0.4363 - accuracy: 0.7974 - val_loss: 0.5320 - val_accuracy: 0.7516 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 67s 337ms/step - loss: 0.3986 - accuracy: 0.8176 - val_loss: 0.5264 - val_accuracy: 0.7366 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 67s 337ms/step - loss: 0.3402 - accuracy: 0.8495 - val_loss: 0.5309 - val_accuracy: 0.7609 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 67s 338ms/step - loss: 0.2789 - accuracy: 0.8809 - val_loss: 0.6089 - val_accuracy: 0.7459 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

...

Epoch 1/20
200/200 [==============================] - 107s 518ms/step - loss: 0.5861 - accuracy: 0.7037 - val_loss: 0.5403 - val_accuracy: 0.7553 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 103s 515ms/step - loss: 0.5087 - accuracy: 0.7586 - val_loss: 0.5170 - val_accuracy: 0.7513 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 103s 514ms/step - loss: 0.4753 - accuracy: 0.7737 - val_loss: 0.5236 - val_accuracy: 0.7325 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 103s 513ms/step - loss: 0.4568 - accuracy: 0.7860 - val_loss: 0.4786 - val_accuracy: 0.7706 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 102s 512ms/step - loss: 0.4377 - accuracy: 0.7941 - val_loss: 0.4878 - val_accuracy: 0.7694 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 102s 513ms/step - loss: 0.4084 - accuracy: 0.8127 - val_loss: 0.5086 - val_accuracy: 0.7522 - lr: 0.0010
Epoch 7/20
200/200 [==========================

2024/10/13 12:58:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp68qb22xd\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp68qb22xd\model\data\model\assets
2024/10/13 12:59:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'leakyrelu', 'dropout_rate': 0.5, 'lstm_units': 64} avec une accuracy de 0.7831
Meilleur modèle entraîné avec les paramètres : {'activation_type': 'leakyrelu', 'dropout_rate': 0.5, 'lstm_units': 64}


Dans notre cas pour comparer les résultats nous allons le faire sous le ui de mlflow.